In [1]:
## IMPORT RELEVANT LIBRAIES

# DATA WRANGLING AND ANALYSIS
import pandas   as pd
import datetime as dt
import numpy    as np
from   skimpy   import skim
from   random   import choice 
#import missingno

# DATA VISULIZATION 
import seaborn           as sns
import matplotlib.pyplot as plt

# DATE MANIPULATION
#from dateutil.relativedelta import relativedelta

In [3]:
# import relevant datasets 

# CUSTOMER DEMOGRAPHIC DATASET
cusDemo    = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 3 , parse_dates=True)
#Transation DATA
transc     = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 1, parse_dates=True) 
#CUSTOMER ADDRESS
cusAddress = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 4, parse_dates=True) 

C:\Users\Owner\AppData\Local\Temp\ipykernel_5112\503302315.py:4: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  cusDemo    = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 3 , parse_dates=True)


#### DATA CLEANING - CUSTOMER DEMOGRAPHICS

In [4]:
cusDemo_cleaned              = cusDemo.copy() 
cusDemo_cleaned.drop(columns = ['first_name', 'last_name', 'default'], inplace = True)

#Clean up gender column
cusDemo_cleaned.gender = df['Gender'].replace({ 'Male': 'M',
                                                'Female': 'F',
                                                'Femal': 'F'})

y  = 2017    
cusDemo_cleaned.DOB.fillna(cusDemo_cleaned.DOB.mode()[0], inplace = True)
cusDemo_cleaned.DOB = round(y - cusDemo_cleaned.DOB.dt.year).astype(int)
cusDemo_cleaned     = cusDemo_cleaned[cusDemo_cleaned.DOB < 100]

#GROUP AGE(Countinous) into discrete variables 
bins    = np.arange(7.5,97.5,5)
labels_ = np.arange(10,95,5)
cusDemo_cleaned['AgeClass'] = pd.cut(cusDemo_cleaned.DOB, bins, labels = labels_)
#cusDemo_cleaned[['AgeClass', 'DOB']]

# Function to fill missing values with random integers between 1 and 22
def fill_missing_with_random_row(row):
    return row.apply(lambda x: np.random.randint(1, 22) if pd.isnull(x) else x)

# Apply the function to the specified row
cusDemo_cleaned.tenure = fill_missing_with_random_row(cusDemo_cleaned.tenure)


"""Convert thsis statement direclty to a pandas dataframe instead of converting to csv first"""
#cusDemo_cleaned[["wealth_segment","job_title","job_industry_category"]].groupby(["wealth_segment","job_industry_category"]).value_counts().to_csv("check_dist.csv")
dictw = pd.read_csv("check_dist.csv")

dictw[(dictw.wealth_segment == "Affluent Customer") 
      & (dictw.job_title == "Editor")].job_industry_category.to_list()

# Function to fill missing values in column C
jic_all = list(dictw.job_industry_category.unique())
cusDemo_cleaned 

def fill_missing_values(cusDemo_cleaned):
    for index, row in cusDemo_cleaned.iterrows():
        if pd.isnull(row['job_industry_category']):
            jic = dictw[(dictw.wealth_segment == row['wealth_segment']) & (dictw.job_title == row['job_title'])].job_industry_category.to_list()
            if len(jic) < 1:
                 cusDemo_cleaned.at[index, 'job_industry_category'] = choice(jic_all)
            else :
                cusDemo_cleaned.at[index, 'job_industry_category'] = choice(jic)
    return cusDemo_cleaned

# Apply the function to the DataFrame
cusDemo_cleaned = fill_missing_values(cusDemo_cleaned)

#Yet to be tested
cusDemo_cleaned.job_title.fillna("Unspecified", inplace = True)

In [16]:
cusDemo_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3999 entries, 0 to 3999
Data columns (total 11 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   customer_id                          3999 non-null   int64   
 1   gender                               3999 non-null   object  
 2   past_3_years_bike_related_purchases  3999 non-null   int64   
 3   DOB                                  3999 non-null   int32   
 4   job_title                            3999 non-null   object  
 5   job_industry_category                3999 non-null   object  
 6   wealth_segment                       3999 non-null   object  
 7   deceased_indicator                   3999 non-null   object  
 8   owns_car                             3999 non-null   object  
 9   tenure                               3999 non-null   float64 
 10  AgeClass                             3999 non-null   category
dtypes: category(1), f

In [ ]:
cusDemo_cleaned = cusDemo.copy() 

##### DROP IRRELEVANT COLUMNS
'''
Drop  default column, i doesnt have a meaning
Drop  default first and last names, Name will not influence purchase pattern
'''
cusDemo_cleaned.drop(columns = ['first_name', 'last_name', 'default'], inplace = True)
#### CLEAN DATE OF BIRTH COLUMN
""" 
- Convert Date of birth to Age 
- Drop null values
- Find and drop outliers
"""

# CONVERT DATE OF BIRTH TO AGE AND CLASSIFY IN TENS 
#take the maximum transcatodedate as the time of the project 
y  = 2017    
"""Fill missing age with mode"""
cusDemo_cleaned.DOB.fillna(cusDemo_cleaned.DOB.mode()[0], inplace = True)
cusDemo_cleaned.DOB = round(y - cusDemo_cleaned.DOB.dt.year).astype(int)
#It seen there is an outlier Age, Lets find out if there are more than one customer that is more than  100 years old 

"""
Oops!, her just caught an oulier.
In a real life situation, we can verify the customer age bu in this case, i will drop the customer from our database 
and the null date of births
"""
cusDemo_cleaned = cusDemo_cleaned[cusDemo_cleaned.DOB < 100]
"""Classify age into bins"""

# Clean job_title','job_industry_category Columns
""""
- Find cell is missing values
- Drop nulls in both category ---> 'job_title','job_industry_category'
"""

cusDemo_cleaned.job_industry_category.isna().value_counts()
cusDemo_cleaned.dropna(how = 'any')

# Attain unique job title to job industry 
job_to_title = cusDemo_cleaned[['job_title','job_industry_category']].dropna().drop_duplicates().reset_index(drop = True)

job_to_title[job_to_title.job_title == 'Blank']
              
test = cusDemo[['customer_id','DOB']].dropna()
test['AGE'] = 2017 - test.DOB.dt.year
test.AGE.astype(int)

test = test[test != test.AGE.max()]

bins = np.arange(10,100,10)
labels_ = np.arange(10,90,10)
test['Grouping'] = pd.cut(test.AGE, bins, labels = labels_ )
test


#### DATA CLEANING - address data 

In [6]:
cusAddress_cleaned = cusAddress.drop(["country"],axis =1) 
cusAddress_cleaned.state[cusAddress_cleaned.state == "New South Wales"] = "NSW"
cusAddress_cleaned.state[cusAddress_cleaned.state == "Victoria"] = "VIC"

C:\Users\Owner\AppData\Local\Temp\ipykernel_2812\191615168.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cusAddress_cleaned.state[cusAddress_cleaned.state == "New South Wales"] = "NSW"
C:\Users\Owner\AppData\Local\Temp\ipykernel_2812\191615168.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cusAddress_cleaned.state[cusAddress_cleaned.state == "Victoria"] = "VIC"


#### DATA MERGING  - address and demographics data 

In [7]:
customer_data = cusDemo_cleaned.merge(cusAddress_cleaned, how = "inner", on = "customer_id")

#### DATA CLEANING - transcation data 

In [36]:
# Copy the transaaction data 
trans_cleaned = transc.copy()

# Investigate if there multiple transactions recorded 
trans_cleaned.transaction_id.duplicated().value_counts()
"""No duplicate transaction Id"""

# Convert product_first_sold_date to the right format 
trans_cleaned['product_first_sold_date'] = pd.to_datetime(trans_cleaned['product_first_sold_date'],
                                           unit = 'D',
                                           origin ='1899-12-30' )

#Convert online type to Boolean
def changeBoolean(var):
    if   var == 0.0:
        return False
    elif var == 1.0:
        return True
    else           :
        return True 
    
trans_cleaned['online_order'] = trans_cleaned['online_order'].apply(changeBoolean)
trans_cleaned.dropna(inplace= True)

In [ ]:
# ADD a profit column and profit ratio
trans_cleaned['profit']      = trans_cleaned['list_price'] - trans_cleaned['standard_cost']
trans_cleaned['profitRatio'] = trans_cleaned['list_price'] / trans_cleaned['standard_cost']

# drop columns with null profit 
trans_cleaned          = trans_cleaned[trans_cleaned['profit'].notna()]

## ADD AN UNSPECIFIED COLUMN TO online_order that is na
trans_cleaned.online_order.fillna('Unspecified', inplace=True)

# Find all categorical columns 
columns = [i for i in list(trans_cleaned.columns) if trans_cleaned[i].dtype == 'O']

for item in columns:
    print(f'{item} unique Values {trans_cleaned[item].unique()}')
    
# Observe thatcertain transactions were cancelled and note that Cancelled transactions dont add up to the revenue
trans_cleaned.order_status.value_counts() 

# Use only approved transcations 
trans_cleaned = trans_cleaned[trans_cleaned.order_status == 'Approved']

#### THE  RELEVANT METRICS I USE WILL BE 
- TOTAL AND AVERAGE REVENUE PER CUSTOMER 
- FREQUENCY OF TRANSCATION 
- AVERAGE PROFIT RATION  PER TRANSACTION 

In [53]:
transcMetrics = trans_cleaned.groupby('customer_id').agg({'profit' : ['sum', 'mean'],
                                          "customer_id" : 'count' ,
                                          'profitRatio' : ['sum', 'mean']})

#### DATA CLEANING
ALL DATA SETS ARE CLEANED INORDER TO BE PREPROCESSED.
REDUNDANT COLUMNS ARE DROPPED, MISSING VALUES ARE FILLED, CATEGORIES ARE CORRECTED. 

#### DATA WRANGLING
DATA IS WRANGLED TO FIT INTO ONE ONE DATASET 

#### COMPUTATION
- COMPUTE AGE AND CLASSIFY BY AGE BRACKET

- AN APPROPRIATE METHOD IS USED TO CLASSIFY CURRENT TARGET- 1,0 WHICH WILL CORRESEPOND TO TRUE OR FALSE
  PROBLEM?? -- EHAT METHOD CAN I USE TO DESIGN A MODEL THAT CAN EFFECTIVELY CATEGORISE TH

#### VISUALIZATION AND CLASSIFICATION OF PUCAHSE BEHAVIOURS 
EDA IS CARRIED OUT O THE EXISTING DATASET AND MORE UNDERSTANDING OF THE PURCHASE BEHAVIOURS IS ATTAINED 

#### FEATURE ENGINEERING 
THE DATA SET IS ACCUSTOMED TO BE FIT INTO A MODEL

#### DEVELOP MODEL 
IN THIS CASE, WE WILL USE A CLASSICAL CLASSIFICTION MODEL TO CLASSIFY CUSKTOMERS INTO TWO GROUPS BASED ON THEIR PURCHASE BEHAVIOUS 

#### MODEL TESTING AND VALIDATION 
THE MODEL IS TESTED WITH THE TEST DATA TO VALIDATE THE CORRECTIES NESSA ND ACCUERACY

#### MODEL DEPLOYMENT
APPLY MODEL ON THE NEW CUSTOMERS FOR CLASSIFICATION